http://pythonhosted.org/svgwrite/

http://web.mit.edu/music21/doc/moduleReference/index.html

In [1]:
from music21 import *

import svgwrite as sw

from IPython.display import display, HTML, Image

import seaborn
from colorsys import hls_to_rgb
from base64 import b16encode

In [11]:
def x_by_cell(cell_num, line_height):
    return round((cell_num) * line_height)

def y_by_line(line_num, cell_width):
    return round((line_num) * cell_width)

def pos_center(x,y,w,h):
    x = (w/2) + x
    y = (h/2) + y

    return (x,y)

svg_attrs = {
    'font-family': 'Inconsolata, monospace',
    'font-weight': 'bold',
    'fill': '#fff',
}

table_attrs = {
    'stroke' : '#000',
    'fill' : '#fff',
    'stroke-width' : '1',
}

cell_attrs = {
    'fill':'none',
    'stroke':'none',
    'stroke-width':'0',
}

pos_attrs_template = {
    'stroke' : '#000',
    'fill' : '#fff',
    'stroke-width' : '1',
    'fill-opacity':'0.6',
}

pos_colors = {
    0: '#000',
    1: '#336',
    2: '#363',
    3: '#f63',
    4: '#c33',
    5: '#6c6',
    6: '#66c',
    7: '#c36',
    8: '#996',
}

pos_attrs = dict()
for p in range(0,9):
    pos_attrs.update({p: {
    'stroke' : '#000',
    #'fill' : pos_colors[p],
    'fill' : '#fff',
    'stroke-width' : '1',
    'fill-opacity':'0.6',
    }})

pos_text_attrs = {
    'font-family': 'Inconsolata, monospace',
    'alignment-baseline' : 'middle',
    'text-anchor' : 'middle',
    'fill' : '#000',
    'stroke-width' : '0',
    'font-weight': 'normal',
}
openstr_attrs = {
    'fill':'none',
    'stroke':'#000',
    'stroke-width':'3',
    
}

closestr_attrs = {
    'fill':'none',
    'stroke':'#000',
    'stroke-width':'1',
}

circ_attrs = {
    'fill':'#fff',
    'fill-opacity': '0.7',
    'stroke':'#000',
    'stroke-width':'1',
}

fretmark_attrs = {
    'fill':'#333',
    'stroke':'#333',
    'stroke-width':'1',
}

text_attrs = {
    'font-family': 'Inconsolata, monospace',
    'alignment-baseline' : 'middle',
    'text-anchor' : 'middle',
    'fill' : '#000',
    'stroke' : 'none',
}

string_num = 4
string_notes = 20 # open string note also counts

width = 970
cell_width = width/string_notes
cell_height = (cell_width/3)*2
height = cell_height*6

dwg = sw.Drawing(profile='full', size=("{}".format(width), "{}".format(height)), **svg_attrs)

dwg.add(sw.shapes.Rect(insert=(0, y_by_line(1, cell_height)), size=(width, cell_height*4), **table_attrs, **{'class':'table-attrs'}))

chromatic = scale.ChromaticScale()

nts = [ chromatic.getPitches('e4', 'b5'),
        chromatic.getPitches('a3', 'e5'),
        chromatic.getPitches('d3', 'a4'),
        chromatic.getPitches('g2', 'd4'),
       ]


pal_hls = seaborn.hls_palette(7, l=.4, s=1).as_hex()

note_colors = dict()
note_names = ['c','d','e','f','g','a','b']

for idx,note_name in enumerate(note_names):
    note_colors.update({note_name.upper() : pal_hls[idx]})

print(note_colors)

#this draws lines for each fret in fretboard
positions = range(1, 20)

for pos_begin in positions:

    # equal for both
    line_x = pos_begin * cell_width
    upper_y = y_by_line(1, cell_height)

    #lower_y = height
    lower_y = y_by_line(1, cell_height) +  (cell_height*4)

    #this allows us to drawn in the v-middle between h-lines
    half_line = cell_height/2


    if pos_begin == 1:
        attrs = openstr_attrs
    else:
        attrs = closestr_attrs

    grp = sw.container.Group()

    positions_start = [1,3,5,7,8,10,12,14]
    if pos_begin in positions_start:

        start_x = pos_begin * cell_width
        start_y = upper_y = y_by_line(1, cell_height)
        
        idx = positions_start.index(pos_begin)
        grp.add(sw.shapes.Rect(insert=(start_x, start_y), size=(cell_width, cell_height*string_num), **pos_attrs[idx+1],**{'class':'pos-highlight pos-highlight-{}'.format(pos_begin)})),

    # this draws the fretboard marks
    fretmarks = [5,7,9,12]
    if pos_begin in fretmarks:

        if pos_begin != 12:
            pos_start_x = pos_begin*cell_width
            pos_start_y = y_by_line(1, cell_height)
            pos_middle_x = pos_start_x + (cell_width/2)
            pos_middle_y = pos_start_y + ((cell_height*string_num)/2)
            circle_radius = ((cell_height/2)/10) * 5

            grp.add(sw.shapes.Circle(center=(pos_middle_x, pos_middle_y), r=circle_radius, **fretmark_attrs, **{'class':'fretmark fretmark-pos-{}'.format(pos_begin)})),
        else:
            # fretmark at position 12 has two dots
            pos_start_x = pos_begin*cell_width
            pos_start_y = y_by_line(1, cell_height)
            pos_middle_x = pos_start_x + (cell_width/2)
            pos_middle_y1 = pos_start_y + ((cell_height*(string_num/2))/2)
            pos_middle_y2 = (pos_start_y + ((cell_height*string_num)/2)) + cell_height
            circle_radius = ((cell_height/2)/10) * 5

            grp.add(sw.shapes.Circle(center=(pos_middle_x, pos_middle_y1), r=circle_radius, **fretmark_attrs, **{'class':'fretmark fretmark-pos-{}'.format(pos_begin)})),
            grp.add(sw.shapes.Circle(center=(pos_middle_x, pos_middle_y2), r=circle_radius, **fretmark_attrs, **{'class':'fretmark fretmark-pos-{}'.format(pos_begin)})),
            
            
    # vertical position lines
    svg_els = [
        sw.shapes.Line(start=(line_x, upper_y), end=(line_x, lower_y), **closestr_attrs, **{'class':'fret fret-{}'.format(pos_begin)}),
    ]

    for element in svg_els:
        grp.add(element)

    dwg.add(grp)

    svg_els = [

        # third
        sw.shapes.Rect(insert=(x_by_cell(5,cell_width), 0), size=(x_by_cell(6,cell_width), cell_height), **pos_attrs[3],**{'class':'pos-label pos-label-3'}),
        sw.text.Text('3rd position', insert=(pos_center(x_by_cell(5,cell_width), 0, x_by_cell(6,cell_width), half_line)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-3'}),

        # seventh
        sw.shapes.Rect(insert=(x_by_cell(12,cell_width), 0), size=(x_by_cell(6,cell_width), cell_height), **pos_attrs[7],**{'class':'pos-label pos-label-7'}),
        sw.text.Text('7th position', insert=(pos_center(x_by_cell(12,cell_width), 0, x_by_cell(6,cell_width), half_line)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-7'}),

        # first
        sw.shapes.Rect(insert=(x_by_cell(1,cell_width), half_line), size=(x_by_cell(7,cell_width), half_line), **pos_attrs[1],**{'class':'pos-label pos-label-1'}),
        sw.text.Text('1st position', insert=(pos_center(x_by_cell(1,cell_width),half_line,x_by_cell(7,cell_width),half_line)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-1'}),

        # fourth
        sw.shapes.Rect(insert=(x_by_cell(7,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(6,cell_width), cell_height), **pos_attrs[4],**{'class':'pos-label pos-label-4'}),
        sw.text.Text('4th position', insert=(pos_center(x_by_cell(7,cell_width), y_by_line(5, cell_height), x_by_cell(6,cell_width), half_line+cell_height)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-4'}),

        #second
        sw.shapes.Rect(insert=(x_by_cell(3,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(6,cell_width), half_line), **pos_attrs[2],**{'class':'pos-label pos-label-2'}),
        sw.text.Text('2nd position', insert=(pos_center(x_by_cell(3,cell_width), y_by_line(5, cell_height), x_by_cell(6,cell_width), half_line)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-2'}),

        # fifth
        sw.shapes.Rect(insert=(x_by_cell(8,cell_width), half_line), size=(x_by_cell(6,cell_width), half_line), **pos_attrs[5],**{'class':'pos-label pos-label-5'}),
        sw.text.Text('5th position', insert=(pos_center(x_by_cell(8,cell_width), half_line, x_by_cell(6,cell_width), half_line)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-5'}),

        # eighth
        sw.shapes.Rect(insert=(x_by_cell(14,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(6,cell_width), cell_height), **pos_attrs[8],**{'class':'pos-label pos-label-8'}),
        sw.text.Text('8th position', insert=(pos_center(x_by_cell(14,cell_width), y_by_line(5, cell_height), x_by_cell(6,cell_width), half_line+cell_height)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-8'}),

        # sixth
        sw.shapes.Rect(insert=(x_by_cell(10,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(6,cell_width), half_line), **pos_attrs[6],**{'class':'pos-label pos-label-6'}),
        sw.text.Text('6th position', insert=(pos_center(x_by_cell(10,cell_width), y_by_line(5, cell_height), x_by_cell(6,cell_width), half_line)), **pos_text_attrs,**{'class':'pos-label-text pos-label-text-6'}),

        # open
        #sw.shapes.Rect(insert=(x_by_cell(0,cell_width), y_by_line(5, cell_height)), size=(x_by_cell(1,cell_width), half_line), **pos_attrs[0]),
        #sw.text.Text('open', insert=(pos_center(x_by_cell(0,cell_width), y_by_line(5, cell_height), x_by_cell(1,cell_width), half_line)), **pos_text_attrs),

    ]

    for element in svg_els:
        grp.add(element)
        
# this loop draws h-lines for the strings
# and also notes for each of them
for string in range(string_num):
    string_y = y_by_line(string+1, cell_height)

    svg_els = [
        sw.shapes.Line(start=(0, string_y), end=(width, string_y), **closestr_attrs,**{'class':'string-v-line string-v-line-{}'.format(string+1)}),
    ]
    grp = sw.container.Group()
    
    for element in svg_els:
        grp.add(element)

    dwg.add(grp)
       
    for note in range(string_notes):
        cell_start_x = note*cell_width
        cell_start_y = string_y
        cell_middle_x = cell_start_x + (cell_width/2)
        cell_middle_y = cell_start_y + (cell_height/2)
        circle_radius = ((cell_height/2)/10) * 7
        
        grp = sw.container.Group()
        #if not nts[string][note].accidental:
        #    note_color_attrs = circ_attrs.update({'fill' : note_colors[nts[string][note].step]})
        #else:
        #    note_color_attrs = circ_attrs.update({'fill' : '#777'})

        nt = nts[string][note]
            
        note_cell_classes = {'class':'''\
note-cell \
note-cell-name-{} \
note-cell-step-{} \
note-cell-octave-{} \
note-cell-alter-in-halfsteps-{} \
note-cell-namewithoctave-{}'''.format(
            nt.name,
            nt.step,
            nt.octave,
            nt.alter,
            nt.nameWithOctave
        )}
        
        note_circ_classes = {'class':'''\
note-circ \
note-circ-name-{} \
note-circ-step-{} \
note-circ-octave-{} \
note-circ-alter-in-halfsteps-{} \
note-circ-namewithoctave-{}'''.format(
            nt.name,
            nt.step,
            nt.octave,
            nt.alter,
            nt.nameWithOctave
        )}
        
        note_text_classes = {'class':'''\
note-text \
note-text-name-{} \
note-text-step-{} \
note-text-octave-{} \
note-text-alter-in-halfsteps-{} \
note-text-namewithoctave-{}'''.format(
            nt.name,
            nt.step,
            nt.octave,
            nt.alter,
            nt.nameWithOctave
        )}
        
        svg_els = [
            sw.shapes.Rect(insert=(cell_start_x, cell_start_y), size=(cell_width, cell_height), **cell_attrs,**note_cell_classes),
            sw.shapes.Circle(center=(cell_middle_x, cell_middle_y), r=circle_radius, **circ_attrs,**note_circ_classes),
            sw.text.Text(nts[string][note].unicodeName, insert=(cell_middle_x, cell_middle_y), **text_attrs,**note_text_classes),
        ]
        
        for element in svg_els:
            grp.add(element)
            
        dwg.add(grp)
display(HTML(dwg.tostring()))

{'C': '#cc0c00', 'D': '#ccbb00', 'E': '#2ecc00', 'F': '#00cc81', 'G': '#0068cc', 'A': '#4700cc', 'B': '#cc00a3'}


In [5]:
with open('fretboard.svg','w') as svg_output:
    svg_output.write(dwg.tostring())